# Fake Tables:

Check efficiency plots w.r.t the reference (noringer)

In [1]:
from kepler.pandas.menu       import ElectronSequence as Chain
from kepler.pandas.readers    import load, load_in_loop, load_hdf
from core import Efficiency, restore_efficiencies
from Gaugi import mkdir_p, progressbar
from prettytable import PrettyTable

import numpy as np
import pandas as pd
import collections
import os
import array
import root_numpy
import rootplotlib as rpl
from pprint import pprint
from copy import deepcopy
import gc
from ROOT import kBlack,kBlue,kRed,kAzure,kGreen,kMagenta,kCyan,kOrange,kGray,kYellow,kWhite,TColor,gStyle,TCanvas
import ROOT
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2
GeV = 1000.


Welcome to JupyROOT 6.16/00
Using all sub packages with ROOT dependence
INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

Applying ATLAS style settings...

Applying ATLAS style settings...

Applying ATLAS style settings...


## 1) Read all files:

In [2]:
output_path = 'output/efficiencies'
mkdir_p(output_path)

In [3]:
# create my chain
chains = [
            Chain( "HLT_e17_lhvloose_nod0_noringer_L1EM15VHI", L1Seed = 'L1_EM15VHI' ),
            Chain( "HLT_e28_lhtight_nod0_noringer_ivarloose", L1Seed = 'L1_EM22VHI' ),
            Chain( "HLT_e60_lhmedium_nod0_noringer_L1EM24VHI", L1Seed = 'L1_EM24VHI' ),
            Chain( "HLT_e140_lhloose_nod0_noringer" , L1Seed = 'L1_EM24VHI' ),
]

versions = ['v8','v12']

for v in versions:
    
    chains.extend( [

            Chain( "HLT_e17_lhvloose_nod0_ringer_%s_L1EM15VHI"%v, L1Seed = 'L1_EM15VHI' , l2calo_column = 'ringer_%s_vloose'%v),
            Chain( "HLT_e28_lhtight_nod0_ringer_%s_ivarloose"%v, L1Seed = 'L1_EM22VHI' , l2calo_column = 'ringer_%s_tight'%v ),
            Chain( "HLT_e60_lhmedium_nod0_ringer_%s_L1EM24VHI"%v, L1Seed = 'L1_EM24VHI' , l2calo_column = 'ringer_%s_medium'%v),
            Chain( "HLT_e140_lhloose_nod0_ringer_%s"%v , L1Seed = 'L1_EM24VHI', l2calo_column = 'ringer_%s_loose'%v),

    ])


def emulate( df ):
    for chain in progressbar(chains, prefix='Emulate...'):
        chain.apply(df)

OrderedDict([('L1Seed', 'L1_EM15VHI'),
             ('L2Calo', (14000.0, 'trig_L2_cl_vloose_et12to22')),
             ('L2', 'trig_L2_el_cut_pt15to20'),
             ('EFCalo', 17000.0),
             ('HLT', (17000.0, 'trig_EF_el_lhvloose'))])
OrderedDict([('L1Seed', 'L1_EM22VHI'),
             ('L2Calo', (25000.0, 'trig_L2_cl_tight_et22toInf')),
             ('L2', 'trig_L2_el_cut_pt20to50'),
             ('EFCalo', 28000.0),
             ('HLT', (28000.0, 'trig_EF_el_lhtight_ivarloose'))])
OrderedDict([('L1Seed', 'L1_EM24VHI'),
             ('L2Calo', (57000.0, 'trig_L2_cl_medium_et22toInf')),
             ('L2', 'trig_L2_el_cut_pt50toInf'),
             ('EFCalo', 60000.0),
             ('HLT', (60000.0, 'trig_EF_el_lhmedium'))])
OrderedDict([('L1Seed', 'L1_EM24VHI'),
             ('L2Calo', (137000.0, 'trig_L2_cl_loose_et22toInf')),
             ('L2', 'trig_L2_el_cut_pt50toInf'),
             ('EFCalo', 140000.0),
             ('HLT', (140000.0, 'trig_EF_el_lhloose'))])
OrderedDic

# 2017:

In [4]:
dpath   = '../data/output/data17_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.dataframe.h5'

In [5]:
data_df = pd.concat( (load_hdf(dpath), 
                      load_hdf('v8/output/emulation/data17_table_v8.h5'),
                      #load_hdf('v9/output/emulation/data17_table_v9.h5'),
                      #load_hdf('v10/output/emulation/data17_table_v10.h5'),
                      #load_hdf('v11/output/emulation/data17_table_v11.h5'),
                      load_hdf('v12/output/emulation/data17_table_v12.h5'),
                      #load_hdf('v13/output/emulation/data17_table_v13.h5'),
                      #load_hdf('v14/output/emulation/data17_table_v14.h5'),
                      #load_hdf('v15/output/emulation/data17_table_v15.h5'),

                     ), axis=1)

In [6]:
emulate(data_df)

Emulate...: 100%|██████████| 12/12 [04:21<00:00, 21.78s/it]


In [7]:
data_df.shape

(43311283, 160)

In [8]:
data_df = data_df.loc[ ((data_df['el_lhmedium']==True) & 
                    (data_df.target==1.0)) | ((data_df.target==0) & 
                    (data_df['el_lhvloose']==False) ) ]

data_df.shape

(39861985, 160)

In [9]:
triggers = [ 
            'e17_lhvloose_nod0_%s_L1EM15VHI', 
            'e28_lhtight_nod0_%s_ivarloose',
            'e60_lhmedium_nod0_%s_L1EM24VHI',
            'e140_lhloose_nod0_%s'
]
pidnames = [
            'el_lhvloose', 
            'el_lhtight', 
            'el_lhmedium',
            'el_lhloose'
]
et_cuts  = [17, 28, 60, 140]
versions = ['noringer','ringer_v8', 'ringer_v12']

for idx, trigger in enumerate(triggers):
    
    t = PrettyTable( ['version', 'FA (L2)', 'FA (HLT)', 'PD (L2)',  'PD (HLT)'] )
    
    is_sgn = (data_df.target==1)&(data_df[pidnames[idx]]==True)&(data_df['el_et']>(et_cuts[idx]+1)*GeV)
    is_bkg = (data_df.target==0)&(data_df['el_lhvloose']==False)&(data_df['el_et']>(et_cuts[idx]+1)*GeV)
    
    print(trigger%'vX')
    for version in versions:
        
            values = [version]
                
            passed = len(data_df.loc[is_bkg&(data_df['L2Calo_'+trigger%version]==True)])
            total  = len(data_df.loc[is_bkg])
            fa = passed/total * 100           
            #values.append( '%1.2f (%d/%d)'%(fa,passed,total) )
            values.append( '%1.2f'%(fa))  


            passed = len(data_df.loc[is_bkg&(data_df['HLT_'+trigger%version]==True)])
            total  = len(data_df.loc[is_bkg])
            fa = passed/total * 100           
            #values.append( '%1.2f (%d/%d)'%(fa,passed,total) )
            values.append( '%1.2f'%(fa))  

            passed = len(data_df.loc[ is_sgn & (data_df['L2Calo_'+trigger%version]==True)])
            total  = len(data_df.loc[is_sgn])
            det = passed/total * 100
            #values.append( '%1.2f (%d/%d)'%(det,passed,total) )
            values.append( '%1.2f'%(det))  


            passed = len(data_df.loc[ is_sgn & (data_df['HLT_'+trigger%version]==True)])
            total  = len(data_df.loc[is_sgn])
            det = passed/total * 100
            #values.append( '%1.2f (%d/%d)'%(det,passed,total) )       
            values.append( '%1.2f'%(det))  

   

            t.add_row(values)
    print(t)


e17_lhvloose_nod0_vX_L1EM15VHI
+------------+---------+----------+---------+----------+
|  version   | FA (L2) | FA (HLT) | PD (L2) | PD (HLT) |
+------------+---------+----------+---------+----------+
|  noringer  |  35.83  |   0.29   |  94.48  |  92.17   |
| ringer_v8  |   2.24  |   0.17   |  94.66  |  92.34   |
| ringer_v12 |   2.14  |   0.16   |  94.65  |  92.34   |
+------------+---------+----------+---------+----------+
e28_lhtight_nod0_vX_ivarloose
+------------+---------+----------+---------+----------+
|  version   | FA (L2) | FA (HLT) | PD (L2) | PD (HLT) |
+------------+---------+----------+---------+----------+
|  noringer  |  26.47  |   0.00   |  93.98  |  86.84   |
| ringer_v8  |   0.84  |   0.00   |  94.22  |  87.01   |
| ringer_v12 |   0.82  |   0.00   |  94.22  |  87.01   |
+------------+---------+----------+---------+----------+
e60_lhmedium_nod0_vX_L1EM24VHI
+------------+---------+----------+---------+----------+
|  version   | FA (L2) | FA (HLT) | PD (L2) | PD (HLT

# 2018:

In [10]:
dpath   = '../data/output/data18_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.dataframe.h5'

In [11]:
data_df = pd.concat( (load_hdf(dpath), 
                      load_hdf('v8/output/emulation/data18_table_v8.h5'),
                      load_hdf('v12/output/emulation/data18_table_v12.h5'),

                     ), axis=1)

In [12]:
emulate(data_df)

Emulate...: 100%|██████████| 12/12 [04:05<00:00, 20.42s/it]


In [13]:
data_df = data_df.loc[ ((data_df['el_lhmedium']==True) & 
                    (data_df.target==1.0)) | ((data_df.target==0) & 
                    (data_df['el_lhvloose']==False) ) ]

data_df.shape

(38543015, 160)

In [14]:
triggers = [ 
            'e17_lhvloose_nod0_%s_L1EM15VHI', 
            'e28_lhtight_nod0_%s_ivarloose',
            'e60_lhmedium_nod0_%s_L1EM24VHI',
            'e140_lhloose_nod0_%s'
]
pidnames = [
            'el_lhvloose', 
            'el_lhtight', 
            'el_lhmedium',
            'el_lhloose'
]
et_cuts  = [17, 28, 60, 140]
versions = ['noringer','ringer_v8', 'ringer_v12']


for idx, trigger in enumerate(triggers):
    
    t = PrettyTable( ['version', 'FA (L2)', 'FA (HLT)', 'PD (L2)',  'PD (HLT)'] )


    is_sgn = (data_df.target==1)&(data_df[pidnames[idx]]==True)&(data_df['el_et']>(et_cuts[idx]+1)*GeV)
    is_bkg = (data_df.target==0)&(data_df['el_lhvloose']==False)&(data_df['el_et']>(et_cuts[idx]+1)*GeV)
    
    print(trigger%'vX')
    for version in versions:
        
            values = [version]
            passed = len(data_df.loc[is_bkg&(data_df['L2Calo_'+trigger%version]==True)])
            total  = len(data_df.loc[is_bkg])
            fa = passed/total * 100           
            #values.append( '%1.2f (%d/%d)'%(fa,passed,total) )
            values.append( '%1.2f'%(fa))  


            passed = len(data_df.loc[is_bkg&(data_df['HLT_'+trigger%version]==True)])
            total  = len(data_df.loc[is_bkg])
            fa = passed/total * 100           
            #values.append( '%1.2f (%d/%d)'%(fa,passed,total) )
            values.append( '%1.2f'%(fa))  

            passed = len(data_df.loc[ is_sgn & (data_df['L2Calo_'+trigger%version]==True)])
            total  = len(data_df.loc[is_sgn])
            det = passed/total * 100
            #values.append( '%1.2f (%d/%d)'%(det,passed,total) )
            values.append( '%1.2f'%(det))  


            passed = len(data_df.loc[ is_sgn & (data_df['HLT_'+trigger%version]==True)])
            total  = len(data_df.loc[is_sgn])
            det = passed/total * 100
            #values.append( '%1.2f (%d/%d)'%(det,passed,total) )       
            values.append( '%1.2f'%(det))  




            t.add_row(values)
    print(t)

e17_lhvloose_nod0_vX_L1EM15VHI
+------------+---------+----------+---------+----------+
|  version   | FA (L2) | FA (HLT) | PD (L2) | PD (HLT) |
+------------+---------+----------+---------+----------+
|  noringer  |  36.80  |   1.02   |  94.45  |  92.34   |
| ringer_v8  |   2.71  |   0.69   |  94.64  |  92.52   |
| ringer_v12 |   2.61  |   0.68   |  94.63  |  92.52   |
+------------+---------+----------+---------+----------+
e28_lhtight_nod0_vX_ivarloose
+------------+---------+----------+---------+----------+
|  version   | FA (L2) | FA (HLT) | PD (L2) | PD (HLT) |
+------------+---------+----------+---------+----------+
|  noringer  |  27.30  |   0.06   |  93.93  |  87.38   |
| ringer_v8  |   1.20  |   0.05   |  94.19  |  87.58   |
| ringer_v12 |   1.18  |   0.05   |  94.20  |  87.60   |
+------------+---------+----------+---------+----------+
e60_lhmedium_nod0_vX_L1EM24VHI
+------------+---------+----------+---------+----------+
|  version   | FA (L2) | FA (HLT) | PD (L2) | PD (HLT